<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/Super_Piano_3_Alt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super Piano 3: Google Music Transformer

In [ ]:
!pip install pretty_midi

In [ ]:
import pickle
import os
import sys
import math
import random
import pickle
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Hyperparameters

In [ ]:
d_model = 512
nhead = 8
dim_feedforward = 1024
dropout = 0.2
num_layer = 6
batch_size = 8
sequence_length = 1024
warmup_steps = 4000
pad_token = 1   
#vocabulary_size = 388 # depends on the dataset

In [ ]:
#@title Functions for MIDI encoder/decoder routines
#Import library for encoder function
import pretty_midi

RANGE_NOTE_ON = 128
RANGE_NOTE_OFF = 128
RANGE_VEL = 32
RANGE_TIME_SHIFT = 100

START_IDX = {
    'note_on': 0,
    'note_off': RANGE_NOTE_ON,
    'time_shift': RANGE_NOTE_ON + RANGE_NOTE_OFF,
    'velocity': RANGE_NOTE_ON + RANGE_NOTE_OFF + RANGE_TIME_SHIFT
}


class SustainAdapter:
    def __init__(self, time, type):
        self.start =  time
        self.type = type


class SustainDownManager:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.managed_notes = []
        self._note_dict = {} # key: pitch, value: note.start

    def add_managed_note(self, note: pretty_midi.Note):
        self.managed_notes.append(note)

    def transposition_notes(self):
        for note in reversed(self.managed_notes):
            try:
                note.end = self._note_dict[note.pitch]
            except KeyError:
                note.end = max(self.end, note.end)
            self._note_dict[note.pitch] = note.start


# Divided note by note_on, note_off
class SplitNote:
    def __init__(self, type, time, value, velocity):
        ## type: note_on, note_off
        self.type = type
        self.time = time
        self.velocity = velocity
        self.value = value

    def __repr__(self):
        return '<[SNote] time: {} type: {}, value: {}, velocity: {}>'\
            .format(self.time, self.type, self.value, self.velocity)


class Event:
    def __init__(self, event_type, value):
        self.type = event_type
        self.value = value

    def __repr__(self):
        return '<Event type: {}, value: {}>'.format(self.type, self.value)

    def to_int(self):
        return START_IDX[self.type] + self.value

    @staticmethod
    def from_int(int_value):
        info = Event._type_check(int_value)
        return Event(info['type'], info['value'])

    @staticmethod
    def _type_check(int_value):
        range_note_on = range(0, RANGE_NOTE_ON)
        range_note_off = range(RANGE_NOTE_ON, RANGE_NOTE_ON+RANGE_NOTE_OFF)
        range_time_shift = range(RANGE_NOTE_ON+RANGE_NOTE_OFF,RANGE_NOTE_ON+RANGE_NOTE_OFF+RANGE_TIME_SHIFT)

        valid_value = int_value

        if int_value in range_note_on:
            return {'type': 'note_on', 'value': valid_value}
        elif int_value in range_note_off:
            valid_value -= RANGE_NOTE_ON
            return {'type': 'note_off', 'value': valid_value}
        elif int_value in range_time_shift:
            valid_value -= (RANGE_NOTE_ON + RANGE_NOTE_OFF)
            return {'type': 'time_shift', 'value': valid_value}
        else:
            valid_value -= (RANGE_NOTE_ON + RANGE_NOTE_OFF + RANGE_TIME_SHIFT)
            return {'type': 'velocity', 'value': valid_value}


def _divide_note(notes):
    result_array = []
    notes.sort(key=lambda x: x.start)

    for note in notes:
        on = SplitNote('note_on', note.start, note.pitch, note.velocity)
        off = SplitNote('note_off', note.end, note.pitch, None)
        result_array += [on, off]
    return result_array


def _merge_note(snote_sequence):
    note_on_dict = {}
    result_array = []

    for snote in snote_sequence:
        # print(note_on_dict)
        if snote.type == 'note_on':
            note_on_dict[snote.value] = snote
        elif snote.type == 'note_off':
            try:
                on = note_on_dict[snote.value]
                off = snote
                if off.time - on.time == 0:
                    continue
                result = pretty_midi.Note(on.velocity, snote.value, on.time, off.time)
                result_array.append(result)
            except:
                print('info removed pitch: {}'.format(snote.value))
    return result_array


def _snote2events(snote: SplitNote, prev_vel: int):
    result = []
    if snote.velocity is not None:
        modified_velocity = snote.velocity // 4
        if prev_vel != modified_velocity:
            result.append(Event(event_type='velocity', value=modified_velocity))
    result.append(Event(event_type=snote.type, value=snote.value))
    return result


def _event_seq2snote_seq(event_sequence):
    timeline = 0
    velocity = 0
    snote_seq = []

    for event in event_sequence:
        if event.type == 'time_shift':
            timeline += ((event.value+1) / 100)
        if event.type == 'velocity':
            velocity = event.value * 4
        else:
            snote = SplitNote(event.type, timeline, event.value, velocity)
            snote_seq.append(snote)
    return snote_seq


def _make_time_sift_events(prev_time, post_time):
    time_interval = int(round((post_time - prev_time) * 100))
    results = []
    while time_interval >= RANGE_TIME_SHIFT:
        results.append(Event(event_type='time_shift', value=RANGE_TIME_SHIFT-1))
        time_interval -= RANGE_TIME_SHIFT
    if time_interval == 0:
        return results
    else:
        return results + [Event(event_type='time_shift', value=time_interval-1)]


def _control_preprocess(ctrl_changes):
    sustains = []

    manager = None
    for ctrl in ctrl_changes:
        if ctrl.value >= 64 and manager is None:
            # sustain down
            manager = SustainDownManager(start=ctrl.time, end=None)
        elif ctrl.value < 64 and manager is not None:
            # sustain up
            manager.end = ctrl.time
            sustains.append(manager)
            manager = None
        elif ctrl.value < 64 and len(sustains) > 0:
            sustains[-1].end = ctrl.time
    return sustains


def _note_preprocess(susteins, notes):
    note_stream = []

    if susteins:    # if the midi file has sustain controls
        for sustain in susteins:
            for note_idx, note in enumerate(notes):
                if note.start < sustain.start:
                    note_stream.append(note)
                elif note.start > sustain.end:
                    notes = notes[note_idx:]
                    sustain.transposition_notes()
                    break
                else:
                    sustain.add_managed_note(note)

        for sustain in susteins:
            note_stream += sustain.managed_notes
    
    else:       # else, just push everything into note stream
        for note_idx, note in enumerate(notes):
            note_stream.append(note)

    note_stream.sort(key= lambda x: x.start)
    return note_stream


def encode_midi(file_path):
    events = []
    notes = []
    mid = pretty_midi.PrettyMIDI(midi_file=file_path)

    for inst in mid.instruments:
        inst_notes = inst.notes
        # ctrl.number is the number of sustain control. If you want to know abour the number type of control,
        # see https://www.midi.org/specifications-old/item/table-3-control-change-messages-data-bytes-2
        ctrls = _control_preprocess([ctrl for ctrl in inst.control_changes if ctrl.number == 64])
        notes += _note_preprocess(ctrls, inst_notes)

    dnotes = _divide_note(notes)

    # print(dnotes)
    dnotes.sort(key=lambda x: x.time)
    # print('sorted:')
    # print(dnotes)
    cur_time = 0
    cur_vel = 0
    for snote in dnotes:
        events += _make_time_sift_events(prev_time=cur_time, post_time=snote.time)
        events += _snote2events(snote=snote, prev_vel=cur_vel)
        # events += _make_time_sift_events(prev_time=cur_time, post_time=snote.time)

        cur_time = snote.time
        cur_vel = snote.velocity

    return [e.to_int() for e in events]


def decode_midi(idx_array, file_path=None):
    event_sequence = [Event.from_int(idx) for idx in idx_array]
    # print(event_sequence)
    snote_seq = _event_seq2snote_seq(event_sequence)
    note_seq = _merge_note(snote_seq)
    note_seq.sort(key=lambda x:x.start)

    mid = pretty_midi.PrettyMIDI()
    # if want to change instument, see https://www.midi.org/specifications/item/gm-level-1-sound-set
    instument = pretty_midi.Instrument(0, False, "Composed by Super Piano Music Transformer AI")
    instument.notes = note_seq

    mid.instruments.append(instument)
    if file_path is not None:
        mid.write(file_path)
    return mid

In [ ]:
!mkdir '/content/midis/'
!mkdir '/content/save/'
!mkdir '/content/train/'
!mkdir '/content/test/'

midi_dir_path = '/content/midis/'
save_dir_path = '/content/save/'
train_dir_path = '/content/train/'
test_dir_path = '/content/test/'
extension = ['.mid', '.midi']

In [ ]:
#@title Download Google Magenta MAESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)
%cd /content/midis
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip
%cd /content

In [ ]:
#@title Encode MIDI files

def encode_midi_files(midi_dir_path, save_dir_path, extension):
  #create directory for saving files
  os.makedirs(save_dir_path, exist_ok=True)
  #get all midi files from midi_directory
  for file in os.listdir(midi_dir_path):
    if file.endswith(tuple(extension)):
        print(os.path.join(midi_dir_path, file))
        print(file + ' is being processed', flush=True)
        try:
          encoded_file = encode_midi(midi_dir_path+file)
        except KeyboardInterrupt:
            print(' Stopped by keyboard')
            return
        except EOFError:
            print('EOF Error')
            return
        with open(save_dir_path+file+'.encoded', 'wb') as f:
            pickle.dump(encoded_file, f)

encode_midi_files(midi_dir_path, save_dir_path, extension) # only use once

In [ ]:
#@title Create and Split MIDI DataSet
# for more convenience we split training data and test data in two different folders
import shutil

def create_dataset(save_dir_path, split_ratio=0.9):
    dataset = [file for file in os.listdir(save_dir_path)]
    np.random.shuffle(dataset)

    train_set = dataset[:int(len(dataset) * split_ratio)]    
    test_set = dataset[int(len(dataset) * split_ratio):]
    
    shutil.rmtree(train_dir_path)
    shutil.rmtree(test_dir_path)

    os.makedirs(train_dir_path, exist_ok=True)
    os.makedirs(test_dir_path, exist_ok=True)

    for file in os.listdir(save_dir_path):
        if os.stat(save_dir_path+file).st_size != 0:
            if file in test_set:
                shutil.copyfile(save_dir_path+file, test_dir_path+file)
            else:
                shutil.copyfile(save_dir_path+file, train_dir_path+file)

create_dataset(save_dir_path, split_ratio=0.9) # only use once

In [ ]:
#@title Setup all functions and procedures

def load_dataset(train_dir_path, test_dir_path):
    #load all encoded file
    train_set = [file for file in os.listdir(train_dir_path)]
    test_set = [file for file in os.listdir(test_dir_path)]

    return train_set, test_set

def generate_batch(dataset, dir_path, sequence_length=1024, batch_size=4):
    sequence_length += 1
    batch_midi = []
    while len(batch_midi) < batch_size:
        file = random.choice(dataset)
        #if the midi contains more sequence that the sequence length
        try:
            with open(dir_path+file, 'rb') as f:
                data = pickle.load(f)
        except:
            print(dir_path+file + " file not found.")
        if sequence_length <= len(data):
            begin_index = random.randrange(0, len(data) - sequence_length)
            data = data[begin_index:begin_index + sequence_length]
            batch_midi.append(data)
    batch_midi = torch.Tensor(batch_midi)
    inputs = batch_midi[:, :-1]
    labels = batch_midi[:, 1:]
    return inputs, labels


class MusicMultiheadAttention(torch.nn.MultiheadAttention):
    def __init__(self, embed_dim, nhead, dropout=0.1, bias=True, add_bias_kv=False, 
                 add_zero_attn=False, kdim=None, vdim=None):
        
        torch.nn.MultiheadAttention.__init__(self, embed_dim, nhead, dropout=0.1, 
                                             bias=True, add_bias_kv=False, 
                                             add_zero_attn=False, kdim=None, vdim=None)
        
        self.embed_dim = embed_dim
        self.weights_q = torch.nn.Linear(self.embed_dim, self.embed_dim)
        self.weights_k = torch.nn.Linear(self.embed_dim, self.embed_dim)
        self.weights_v = torch.nn.Linear(self.embed_dim, self.embed_dim)
        self.weights_o = torch.nn.Linear(self.embed_dim, self.embed_dim)

    def forward(self, query, key, value, key_padding_mask=None,
                need_weights=True, attn_mask=None):
        Q, K, V = self.transform_input(query, key, value)
        # Reshaping the matrices 
        # Each L × D query, key, and value matrix is then split into H L × D 
        # h_D parts or attention heads, indexed by h, and with dimension D_h = D/H
        Q = self.matrix_to_heads(Q)
        K = self.matrix_to_heads(K)
        V = self.matrix_to_heads(V)
        
        # learning a separate relative position embedding Er of shape (H, L, Dh)
        Er = torch.randn([self.num_heads, query.size(1), self.head_dim], requires_grad=False).to(device)

        # we transpose the two last dimensions of Er to realize Q*Er^T 
        QEr = torch.matmul(Q, torch.transpose(Er,1,2))
        # QEr of shape (B, H, L, L)     
        # QEr = torch.einsum('bhld,ld->bhll', [Q, Er])

        # 1. Pad a dummy column vector of length L before the leftmost column.
        QEr = torch.nn.functional.pad(QEr, (1,0), mode="constant", value=0)

        # 2. Reshape the matrix to have shape (L+1, L). 
        QEr = torch.reshape(QEr, [QEr.size(0), QEr.size(1), QEr.size(3), QEr.size(2)])
        
        # 3. Slice that matrix to retain only the last l rows and all the columns, 
        # resulting in a (L, L) matrix again, but now absolute-by-absolute indexed, 
        # which is the S rel that we need.
        S_rel = QEr[:,:,1:,:]

        z_attention = self.attention(Q, K, V, S_rel, attn_mask)
        z_attention = self.weights_o(z_attention)
        # Masking can be added and Dropout ?

        return z_attention

    def attention(self, Q, K, V, S, mask):
        # Dh = self.head_dim // self.num_heads
        logits = torch.add(torch.matmul(Q, torch.transpose(K, 2, 3)), S) / math.sqrt((self.head_dim // self.num_heads))
        # print("logits : ", logits.size())
        # print("mask : ", mask.size())
        if mask is not None:
        #    mask = mask.unsqueeze(1) #shape of mask must be broadcastable with shape of underlying tensor
            logits = logits.masked_fill(mask == 0, -1e9) #masked_fill fills elements of scores with -1e9 where mask == 0
        #if mask is not None:
        #    logits += (mask.to(torch.int64) * -1e9).to(logits.dtype)        
            
        activation = F.softmax(logits, -1)
        attention = torch.matmul(activation, V)
        attention = torch.reshape(attention, (attention.size(0), -1, self.embed_dim))
        return attention
    
    def matrix_to_heads(self, qkv):
        '''
            Takes a query/key/value (qkv) matrix and reshapes it to  B * H * L * D_h heads 
            with dimension D_h = D/H
        '''
        batch_size_q = qkv.size(0)
        #qkv = torch.reshape(qkv, (batch_size_q, qkv.size(0), self.num_heads, self.head_dim))
        qkv = torch.reshape(qkv, (batch_size_q, self.num_heads, qkv.size(1), self.head_dim))
        return qkv

    def transform_input(self, query, key, value):
        '''
            Transforming the input vector, X, of LxD dimension 
            into 
                queries: Q = XW^Q 
                keys:    K = XW^K
            and values:  V = XW^V
            which are all DxD square matrices.
        '''
        return self.weights_q(query), self.weights_k(key), self.weights_v(value)
    

class MusicTransformerEncoderLayer(torch.nn.TransformerEncoderLayer):
    def __init__(self, d_model, nhead, dim_feedforward=2048,
                 dropout=0.1, activation="relu"):
        torch.nn.TransformerEncoderLayer.__init__(self, d_model, nhead)
        self.d_model = d_model
        # OverRide
        self.self_attn = MusicMultiheadAttention(d_model, nhead)

class MusicTransformerDecoderLayer(torch.nn.TransformerDecoderLayer):
    def __init__(self, d_model, nhead, dim_feedforward=2048,
                 dropout=0.1, activation="relu"):
        torch.nn.TransformerDecoderLayer.__init__(self, d_model,nhead)
        self.d_model = d_model
        # OverRide
        self.self_attn = MusicMultiheadAttention(d_model, nhead)

class MusicTransformerEncoder(torch.nn.TransformerEncoder):
    def __init__(self, encoder_layer, vocabulary_size=390, num_encoder_layers=6, normalization=None):
        super().__init__(encoder_layer, num_encoder_layers, normalization)
        self.d_model = encoder_layer.d_model
        self.vocabulary_size = vocabulary_size
        self.dropout = torch.nn.Dropout(encoder_layer.dropout.p)
        self.embedding = torch.nn.Embedding(num_embeddings=self.vocabulary_size, embedding_dim=self.d_model)
    
    def forward(self, src, mask=None, src_key_padding_mask=None):
        
        pos_encoding = DynamicPositionEmbedding(self.d_model, src.size(1))
        
        src = math.sqrt(self.d_model) * self.embedding(src.to(torch.long).to(device))
        src = pos_encoding(src)
        src = self.dropout(src)

        return super().forward(src, mask=mask, src_key_padding_mask=src_key_padding_mask)


class MusicTransformerDecoder(torch.nn.TransformerDecoder):
    def __init__(self, decoder_layer, vocabulary_size=390, num_decoder_layers=6, normalization=None):
        super().__init__(decoder_layer, num_decoder_layers, normalization)
        self.d_model = decoder_layer.d_model
        self.vocabulary_size = vocabulary_size
        self.dropout = torch.nn.Dropout(decoder_layer.dropout.p)
        self.embedding = torch.nn.Embedding(num_embeddings=self.vocabulary_size, embedding_dim=self.d_model)

    def forward(self, tgt, memory, tgt_mask=None, 
                memory_mask=None, tgt_key_padding_mask=None,
                memory_key_padding_mask=None):        
                
        pos_encoding = DynamicPositionEmbedding(self.d_model, tgt.size(1))

        tgt = pos_encoding(math.sqrt(self.d_model) * self.embedding(tgt.to(torch.long).to(device)))
        tgt = self.dropout(tgt)

        return super().forward(tgt, memory, tgt_mask=tgt_mask, 
                memory_mask=memory_mask, tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=memory_key_padding_mask)


class MusicTransformer(torch.nn.modules.Transformer):
    def __init__(self, d_model=512, nhead=8, vocabulary_size=388,
                 num_encoder_layers=6, num_decoder_layers=6, 
                 dim_feedforward=2048, dropout=0.1, activation="relu", 
                 custom_encoder=None, custom_decoder=None):
        
        super().__init__(d_model=d_model, nhead=nhead, 
                         num_encoder_layers=num_encoder_layers,
                         num_decoder_layers=num_decoder_layers, 
                         dim_feedforward=dim_feedforward, dropout=dropout, activation=activation, 
                         custom_encoder=custom_encoder, custom_decoder=custom_decoder)
        
        self.vocabulary_size = vocabulary_size
        ###        
        self.fc = torch.nn.Linear(self.d_model, self.vocabulary_size)
        self._reset_parameters()

    def forward(self, src, tgt, src_mask=None, tgt_mask=None,
                memory_mask=None, src_key_padding_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):

        #if src.size(1) != tgt.size(1):
        #    raise RuntimeError("the batch number of src and tgt must be equal")

        #if src.size(2) != self.d_model or tgt.size(2) != self.d_model:
        #    raise RuntimeError("the feature number of src and tgt must be equal to d_model")
            
        memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)

        output = self.fc(output)        
        
        return output




class DynamicPositionEmbedding(torch.nn.Module):
    def __init__(self, embedding_dim, max_seq=1024):
        super().__init__()
        embed_sinusoid_list = np.array([[
            [
                math.sin(
                    pos * math.exp(-math.log(10000) * i/embedding_dim) *
                    math.exp(math.log(10000)/embedding_dim * (i % 2)) + 0.5 * math.pi * (i % 2)
                )
                for i in range(embedding_dim)
            ]
            for pos in range(max_seq)
        ]])
        self.positional_embedding = embed_sinusoid_list

    def forward(self, x):
        x = x + torch.from_numpy(self.positional_embedding[:, :x.size(1), :]).to(x.device, dtype=x.dtype)
        return x


class PositionalEncoder(torch.nn.Module):
    def __init__(self, d_model, max_seq_len=1024):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                    math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i + 1] = \
                    math.cos(pos / (10000 ** ((2 * (i + 1)) / d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        with torch.no_grad():
            x = x * math.sqrt(self.d_model)
            seq_len = x.size(1)
            self.pe.to(device)
            print("self.pe : ", self.pe.device.type)
            pe = self.pe[:, :seq_len]
            print("pe : ", pe.device.type)
            x = x + pe
            return x   

In [ ]:
#@title More functions
from torch.utils.data.dataset import Dataset

def tensorFromSequence(sequence):
    """
    Generate tensors from the sequence in numpy.
    """
    output = torch.tensor(sequence).long()

    return output


def PrepareData(npz_file, split='train', L=1024):
    """
    Function to prepare the data into pairs (input, target).
    Adds [PAD], [SOS] and [EOS] tokens into the data,
    where [PAD]=1, [SOS]=2, [EOS]=3.
    Limits the sequence to length of L.
    """
    print("Preparing data for",split,"split...")
    # Load in the data
    full_data = np.load(npz_file, fix_imports=True, encoding="latin1", allow_pickle=True)
    data = full_data[split]

    # Extract the vocab from file
    vocab = GenerateVocab(npz_file)
    # Generate new vocab to map to later
    new_vocab = np.arange(len(vocab))

    # Initialize the tokens
    pad_token = np.array([[1]])

    # Repeat for all samples in data
    pairs = []
    for samples in data:
        # Serialise the dataset so that the resulting sequence is
        # S_1 A_1 T_1 B_1, S_2 A_2 T_2 B_2, ...

        # Generate input
        input_seq = samples.flatten()

        # Cut off the samples so that it has length of 1024
        if(len(input_seq) >= L):
            # input_seq = input_seq[:L-1]
            input_seq = input_seq[:L]

        # Set the NaN values to 0 and reshape accordingly
        input_seq = np.nan_to_num(input_seq.reshape(1,input_seq.size))

        # Generate target
        output_seq = input_seq[:,1:]

        # For both sequences, pad to sequence length L
        pad_array = pad_token * np.ones((1,L-input_seq.shape[1]))
        input_seq = np.append(input_seq, pad_array,axis=1)
        pad_array = pad_token * np.ones((1,L-output_seq.shape[1]))
        output_seq = np.append(output_seq, pad_array,axis=1)

        # Map the pitch value to int values below vocab size
        for i, val in enumerate(vocab):
            input_seq[input_seq==val] = new_vocab[i]
            output_seq[output_seq==val] = new_vocab[i]

        # Make it into a pair
        pair = [input_seq, output_seq]

        # Combine all pairs into one big list of pairs
        pairs.append(pair)

    print("Generated data pairs.")
    return np.array(pairs)

def GenerateVocab(npz_file):
    """
    Generate vocabulary for the dataset including the custom tokens.
    """
    full_data = np.load(npz_file, fix_imports=True, encoding="latin1", allow_pickle=True)
    train_data = full_data['train']
    validation_data = full_data['valid']
    test_data = full_data['test']

    combined_data = np.concatenate((train_data, validation_data, test_data))

    vocab = np.nan
    for sequences in combined_data:
        vocab = np.append(vocab,np.unique(sequences))

    vocab = np.unique(vocab)
    vocab = vocab[~np.isnan(vocab)]
    vocab = np.append([0,1],vocab)
    return vocab 
 

def batched_learning(train,batch_size):
    for i in range(0, len(train), batch_size):
        train1 = train[i:i + batch_size]
        yield train1[:,0],train1[:,1]


##3 - Training

In [ ]:
#@title Definitions and Functions
 
# vocabulary_size depends on the midi encodding
# ~> 388(+2) for encoded_midi / epiano compt 
# ~> 46(+2) for encoded_midi / epiano compt                 

# DEFINING THE MODEL

vocabulary_size = 390

normalization = torch.nn.LayerNorm(d_model)

custom_encoder_layer = MusicTransformerEncoderLayer(d_model=d_model, nhead=nhead, 
                                               dim_feedforward=dim_feedforward, 
                                               dropout=dropout, activation="relu")

custom_decoder_layer = MusicTransformerDecoderLayer(d_model=d_model, nhead=nhead, 
                                               dim_feedforward=dim_feedforward, 
                                               dropout=dropout, activation="relu")

custom_encoder = MusicTransformerEncoder(custom_encoder_layer, vocabulary_size, num_layer, normalization)
custom_decoder = MusicTransformerDecoder(custom_decoder_layer, vocabulary_size, num_layer, normalization)

model = MusicTransformer(d_model=d_model, nhead=nhead, 
                         vocabulary_size=vocabulary_size, 
                         num_encoder_layers=num_layer, 
                         num_decoder_layers=num_layer, 
                         dim_feedforward=dim_feedforward, 
                         dropout=dropout, activation="relu", 
                         custom_encoder=custom_encoder, 
                         custom_decoder=custom_decoder)
# Give model to the current device (hopefully cuda)
model.to(device)

# Optimizer
# Adam optimizer [20] with β 1 = 0.9, β 2 = 0.98 and  = 10 −9
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-09, weight_decay=1e-4)

# Define a scheduler to vary the learning rate

class Scheduler:
    def __init__(self, optimizer, d_model=d_model, warmup_steps=4000):
        self.optimizer = optimizer
        self.d_model = d_model
        self.step_num = 1
        self.l_rate = 0
        self.warmup_steps = warmup_steps

    def step(self):        
        # increment step
        self.step_num += 1

        # compute new learning rate        
        self.l_rate = self.d_model**(-.5) * min(self.step_num**(-.5), self.step_num * self.warmup_steps**(-1.5))

        # update optimizer learning rate
        for p in optimizer.param_groups:
            p['lr'] = self.l_rate

        # update the weights in the network
        self.optimizer.step()


# See if it is possible to do it using lr_scheduler.LambdaLR lr_scheduler.StepLR
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1)
scheduler = Scheduler(optimizer, d_model, warmup_steps)


### Mask Generation from https://github.com/COMP6248-Reproducability-Challenge/music-transformer-comp6248/blob/master/MaskGen.py

# Filename: MaskGen.py
# Date Created: 15-Mar-2019 2:42:12 pm
# Description: Functions used to generate masks w.r.t. given inputs.
from torch.autograd import Variable

def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0).to(device)
    return np_mask


def create_masks(src, trg, pad_token):
    src_mask = (src != pad_token).unsqueeze(-2).to(device)

    if trg is not None:
        trg_mask = (trg != pad_token).unsqueeze(-2).to(device)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size)
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask


def count_nonpad_tokens(target):
    nonpads = (target != 1).squeeze()
    ntokens = torch.sum(nonpads)
    return ntokens      

### Training with data from Midi_Encoded 

In [ ]:
#@title Load the existing model checkpoint or define a new model
train_data, test_data = load_dataset(train_dir_path, test_dir_path)

ckpt_dir = '/content'
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)
  
best_loss = 10.
model_name = 'midi_encoded_6-1'
ckpt_path = '/content/'+model_name+'.pt'
if os.path.exists(ckpt_path):
    ckpt = torch.load(ckpt_path)
    try:
      model.load_state_dict(ckpt['my_model'])
      optimizer.load_state_dict(ckpt['optimizer'])
      best_acc = ckpt['best_loss']
    except RuntimeError as e:
        print('wrong checkpoint')
    else:    
      print('checkpoint is loaded !')
      print('current best loss : %.2f' % best_loss)

train_writer = SummaryWriter()
test_writer = SummaryWriter()

In [ ]:
#@title Train the model
# Training
max_epochs = 100
n_iter = 0

# each 50 iterations we are going to compare the losses
total_train_loss = []
total_valid_loss = []

for e in range(max_epochs):
    model.train()
    train_loss = []
    for b_train in range(len(train_data) // batch_size):
        
        n_iter += 1
        # train phase
        # feed data into the network and get outputs.
        # feed data into the network and get outputs.
        inputs, target = generate_batch(train_data, train_dir_path, sequence_length=sequence_length, batch_size=batch_size)
              
        # Train on GPU
        inputs.to(device)
        target.to(device)
        ys = target.contiguous().view(-1).to(torch.long).to(device)
        
        # Create mask for both input and target sequences
        input_mask, target_mask = create_masks(torch.reshape(inputs, (batch_size, 1, -1)), torch.reshape(target, (batch_size, 1, -1)), pad_token)        
        
        # feed data into the network and get outputs.
        preds_idx = model(inputs, target, input_mask, target_mask)
        
        # Flush out gradients computed at the previous step before computing gradients at the current step. 
        #       Otherwise, gradients would accumulate.
        optimizer.zero_grad()

        # calculate loss
        loss = F.cross_entropy(preds_idx.contiguous().view(preds_idx.size(-1), -1).transpose(0,1), ys, ignore_index = pad_token, size_average = False) / (count_nonpad_tokens(ys))

        # accumulates the gradient and backprogate loss.
        loss.backward()

        # performs a parameter update based on the current gradient
        scheduler.step()    
        
        print('\n====================================================')
        print('Epoch/Batch: {}/{}'.format(e, b_train))
        print('Train >>>> Loss: {:6.6}'.format(loss))

        train_loss.append(loss.item())

    print('\n**************************************************')
    print("\n*** Test *** ")
    
    # Validation step
    model.eval()
    valid_loss = []
    with torch.no_grad():
        for b_test in range(len(test_data) // batch_size):        
            inputs, target = generate_batch(test_data, test_dir_path, sequence_length=sequence_length, batch_size=batch_size)
                  
            # Train on GPU
            inputs.to(device)
            target.to(device)
            ys = target.contiguous().view(-1).to(torch.long).to(device)
            
            # Create mask for both input and target sequences
            input_mask, target_mask = create_masks(torch.reshape(inputs, (batch_size, 1,-1)), torch.reshape(target, (batch_size, 1, -1)), pad_token)        

            # Feed Forward
            preds_validate = model(inputs, target, input_mask, target_mask)
            loss = F.cross_entropy(preds_validate.contiguous().view(preds_validate.size(-1), -1).transpose(0,1), ys, \
                                    ignore_index = pad_token, size_average = False) / (count_nonpad_tokens(ys))
            valid_loss.append(loss.item())

    avg_train_loss = np.mean(train_loss)
    avg_valid_loss = np.mean(valid_loss)

    total_train_loss.append(avg_train_loss)
    total_valid_loss.append(avg_valid_loss)

    print("[Average Train Loss]: {:6.6}".format(avg_train_loss))
    print("[Average Testing Loss]: {:6.6}".format(avg_valid_loss))

    # save checkpoint whenever there is improvement in performance
    if avg_valid_loss < best_loss:
        best_loss = avg_valid_loss
        # Note: optimizer also has states ! don't forget to save them as well.
        ckpt = {'my_model':model.state_dict(),
                'optimizer':optimizer.state_dict(),
                'best_loss':best_loss}
        torch.save(ckpt, ckpt_path)
        print('checkpoint is saved !')

    train_writer.add_scalar('loss/train', avg_train_loss, global_step=n_iter)
    test_writer.add_scalar('loss/valid', avg_valid_loss, global_step=n_iter)
    print('\n**************************************************')

train_writer = SummaryWriter()
test_writer = SummaryWriter()

In [ ]:
#@title Plot loss graphs
import matplotlib.pyplot as plt

plt.plot(total_train_loss, label='train loss')
plt.plot(total_valid_loss, label='validation loss')
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
img_path = os.path.join('/gdrive/My Drive/my_data/library/checkpoints/', "encoded_midi_6-1.png")
plt.savefig("encoded_midi_6-1.png")

### Training with data from JS Bach Chorales 

In [ ]:
#@title Load the existing model checkpoint or define a new model
src_data = '/gdrive/My Drive/my_data/library/Jsb16thSeparated.npz'

# Generate the vocabulary from the data
vocab = GenerateVocab(src_data)
vocabulary_size = len(vocab)
pad_token = 1

# since we change de vocab len the model should be create after this
# we just show this as an exemple of training of JS Bach Chorales dataset 
# thanks to propressing functions from another repository.

# Setup the dataset for training split and validation split
train_data = PrepareData(src_data ,'train', int(sequence_length))
valid_data = PrepareData(src_data ,'valid', int(sequence_length))

ckpt_dir = os.path.join(gdrive_root, '/my_data/library/checkpoints/')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)
  
best_loss = 10.
model_name = 'midi_encoded_6-1'
ckpt_path = '/gdrive/My Drive/my_data/library/checkpoints/train-nlayer_'+model_name+'.pt'
if os.path.exists(ckpt_path):
    ckpt = torch.load(ckpt_path)
    try:
      model.load_state_dict(ckpt['my_model'])
      optimizer.load_state_dict(ckpt['optimizer'])
      best_acc = ckpt['best_loss']
    except RuntimeError as e:
        print('wrong checkpoint')
    else:    
      print('checkpoint is loaded !')
      print('current best loss : %.2f' % best_loss)

In [ ]:
#@title Train the model
# Training
max_epochs = 100
n_iter = 0

# each 50 iterations we are going to compare the losses
total_train_loss = []
total_valid_loss = []

for e in range(max_epochs):
    model.train()
    random.shuffle(train_data)
    train_loss = []
    for b_train, batch in enumerate(batched_learning(train_data, batch_size=batch_size)):        
        
        n_iter += 1
        # train phase
        # feed data into the network and get outputs.
        inputs, target = batch
        
        #print(inputs.shape, target.shape)

        # Train on GPU
        inputs = (tensorFromSequence(inputs)).to(device)
        target = (tensorFromSequence(target)).to(device)

        # Create mask for both input and target sequences
        input_mask, target_mask = create_masks(inputs, target, pad_token)

        inputs = inputs[:,0,:]
        target = target[:,0,:] 

        ys = target.contiguous().view(-1)              

        # ys = labels.contiguous().view(-1).to(torch.long).to(device)
        
        # feed data into the network and get outputs.
        preds_idx = model(inputs, target, input_mask, target_mask)
        
        # Flush out gradients computed at the previous step before computing gradients at the current step. 
        #       Otherwise, gradients would accumulate.
        optimizer.zero_grad()

        # calculate loss
        loss = F.cross_entropy(preds_idx.contiguous().view(preds_idx.size(-1), -1).transpose(0,1), ys, ignore_index = pad_token, size_average = False) / (count_nonpad_tokens(ys))

        # accumulates the gradient and backprogate loss.
        loss.backward()

        # performs a parameter update based on the current gradient
        scheduler.step()    
        
        print('\n====================================================')
        print('Epoch/Batch: {}/{}'.format(e, b_train))
        print('Train >>>> Loss: {:6.6}'.format(loss))

        train_loss.append(loss.item())

    print('\n**************************************************')
    print("\n*** Test *** ")
    
    # Validation step
    model.eval()
    valid_loss = []
    with torch.no_grad():
        pair = valid_data
        inputs = tensorFromSequence(pair[0]).to(device)
        target = tensorFromSequence(pair[1]).to(device)
        
        # Create mask for both input and target sequences
        input_mask, target_mask = create_masks(inputs, target, pad_token)

        inputs = inputs[:,0,:]
        target = target[:,0,:] 
        ys = target.contiguous().view(-1)

        preds_validate = model(inputs, target, input_mask, target_mask)
        loss = F.cross_entropy(preds_validate.contiguous().view(preds_validate.size(-1), -1).transpose(0,1), ys, \
                                ignore_index = pad_token, size_average = False) / (count_nonpad_tokens(ys))
        valid_loss.append(loss.item())

    avg_train_loss = np.mean(train_loss)
    avg_valid_loss = np.mean(valid_loss)

    total_train_loss.append(avg_train_loss)
    total_valid_loss.append(avg_valid_loss)

    print("[Average Train Loss]: {:6.6}".format(avg_train_loss))
    print("[Average Testing Loss]: {:6.6}".format(avg_valid_loss))

    # save checkpoint whenever there is improvement in performance
    if avg_valid_loss < best_loss:
        best_loss = avg_valid_loss
        # Note: optimizer also has states ! don't forget to save them as well.
        ckpt = {'my_model':model.state_dict(),
                'optimizer':optimizer.state_dict(),
                'best_loss':best_loss}
        torch.save(ckpt, ckpt_path)
        print('checkpoint is saved !')

    train_writer.add_scalar('loss/train', avg_train_loss, global_step=n_iter)
    test_writer.add_scalar('loss/valid', avg_valid_loss, global_step=n_iter)
    print('\n**************************************************')
    # torch.cuda.empty_cache()
    # torch.save(model.state_dict(), '/gdrive/My Drive/my_data/library/checkpoints/train-{}.pt'.format(e))
    ckpt = {'my_model':model.state_dict(),
            'optimizer':optimizer.state_dict(),
            'best_loss':best_loss}
    torch.save(model.state_dict(), ckpt_backup_path)
    torch.save(ckpt, ckpt_fullbackup_path)

train_writer = SummaryWriter()
test_writer = SummaryWriter()

# Prediction

In [ ]:
model.load_state_dict(ckpt['my_model'])
optimizer.load_state_dict(ckpt['optimizer'])
best_acc = ckpt['best_loss']

model.eval()

In [ ]:
inputs = torch.randint(0, 1000, (1,10)).to(device)
gen_length = 30
generated_midi = torch.Tensor()
for seq in range(gen_length):
    if seq % 20 == 0: 
        print(seq)
    #print(inputs)
    logits = F.softmax(model(inputs[:, :-1], inputs[:, 1:]), -1).to(device)

    logits = logits[0, :, :]
    
    one_hot = torch.distributions.OneHotCategorical(probs=logits[:,-1])
    res = one_hot.sample().argmax(-1).unsqueeze(-1).to(device)

    #res = torch.transpose(res, 0, 1)

    inputs = torch.cat((inputs[0], res), dim=-1)
    inputs = torch.reshape(inputs, (1, -1))

In [ ]:
from matplotlib import pyplot as pt

x = range(0, len(inputs[0]))
y = [ints.item() for ints in inputs[0]]

p = pt.scatter(x, y, label='generated pitches')

pt.xlabel('index')
pt.ylabel('midi value (pitch)')
pt.show()

In [ ]:
midi_boy = decode_midi(inputs[0])
torch.save(midi_boy, gdrive_root + '/my_data/library/test.midi')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')